<a href="https://colab.research.google.com/github/teamgaon/SANUP/blob/main/220407_sm_SANUP_inference_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [ ]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers
# !pip install git+https://github.com/ssut/py-hanspell.git
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

     |████████████████████████████████| 325 kB 5.2 MB/s 
     |████████████████████████████████| 4.0 MB 40.1 MB/s 
     |████████████████████████████████| 212 kB 83.0 MB/s 
     |████████████████████████████████| 1.1 MB 65.3 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
     |████████████████████████████████| 136 kB 66.3 MB/s 
     |████████████████████████████████| 127 kB 70.4 MB/s 
     |████████████████████████████████| 144 kB 77.6 MB/s 
     |████████████████████████████████| 94 kB 2.2 MB/s 
     |████████████████████████████████| 271 kB 83.3 MB/s 
     |████████████████████████████████| 6.5 MB 72.5 MB/s 
     |████████████████████████████████| 596 kB 84.1 MB/s 
     |████████████████████████████████| 895 kB 80.5 MB/s 
     |████████████████████████████████| 1.2 MB 76.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

- TPU 사용을 위한 accelerate

In [ ]:
from accelerate import Accelerator

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher
from pathlib import Path
# from hanspell import spell_checker
from pykospacing import Spacing

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')
submission = pd.read_csv('/content/drive/MyDrive/SANUP/답안 작성용 파일.csv', encoding='cp949')

In [ ]:
df = df.fillna('')
df['text'] = df['text_obj'].astype(str) + " " + df['text_mthd'].astype(str) + " " + df['text_deal'].astype(str)
df

# 토크나이저, 함수

In [ ]:
def remove_something(text:str):
  return text.replace('&',' ')

df['text'] = df['text'].map(remove_something)

def remove_something(text:str):
  return text.replace('.',' ')

df['text'] = df['text'].map(remove_something)

def remove_something(text:str):
  return text.replace(',',' ')

df['text'] = df['text'].map(remove_something)

In [ ]:
from transformers import AutoTokenizer
from transformers import ElectraModel, ElectraTokenizer

checkpoint = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
spacing = Spacing()

def text_space(text:str):
  return spacing(text)

## 띄어쓰기 전처리

In [ ]:
for i in range(1,11):
  my_file = Path("/content/drive/MyDrive/220407_inference/train"+str(i))
  if my_file.is_file():
    continue
  start = (i-1)*100000
  end = i*100000
  temp = df[start:end]
  temp['text'] = temp['text'].map(text_space)
  temp.to_csv('/content/drive/MyDrive/220407_inference/train'+str(i)+'.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/220407_inference/train1.csv')

for i in range(2, 11):
  temp = pd.read_csv('/content/drive/MyDrive/220407_inference/train'+str(i)+'.csv')
  df = pd.concat([df,temp], axis=0)

In [ ]:
df

In [ ]:
test = test.fillna('')
test['text'] = test['text_obj'].astype(str) + " " + test['text_mthd'].astype(str) + " " + test['text_deal'].astype(str)
test['target'] = test['digit_1'].astype(str) + " " + test['digit_2'].astype(str) + " " + test['digit_3'].astype(str)
test = test[['text', 'target']]

def remove_something(text:str):
  return text.replace('&',' ')

test['text'] = test['text'].map(remove_something)

def remove_something(text:str):
  return text.replace('.',' ')

test['text'] = test['text'].map(remove_something)

def remove_something(text:str):
  return text.replace(',',' ')

test['text'] = test['text'].map(remove_something)

test_original = test

# 학습

* Model
  + Roberta-large

* Optimizer
  + AdamW

* Learning rate scheduler
  + Cosine annealing with warmup

In [ ]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32)
  # eval_dataloader = torch.utils.data.DataLoader(
  #                   train_dataset,
  #                   batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader = accelerator.prepare(model, optimizer, train_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220407_inference/model'+str(i), save_function=accelerator.save)

## digit 별 학습

In [ ]:
for i in range(1,4):
  my_file = Path("/content/drive/MyDrive/220407_inference/model"+str(i))
  if my_file.is_dir():
    continue

  train = df[['text']]
  target = 'digit_'+str(i)
  train['target'] = df[target]

  my_dict = {}
  for i in range(len(train['target'].unique())):
    my_dict[train['target'].unique()[i]] = i

  def target_to_num(target:str):
    return my_dict[target]

  train['target'] = train['target'].map(target_to_num)
  my_dict['  '] = len(my_dict)

  print('--------------------------------')

  tokenized_train = tokenizer(
    list(train['text']),
    return_tensors="pt",
    max_length=64, # Max_Length = 64
    padding=True,
    truncation=True,
    add_special_tokens=True
    )
    
  train_label = train['target']

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

  test = test_original
  
  tokenizer = AutoTokenizer.from_pretrained(checkpoint)

  test_label = test['target'].map(target_to_num)

  tokenized_test = tokenizer(
      list(test['text']),
      return_tensors="pt",
      max_length=64,
      padding=True,
      truncation=True,
      add_special_tokens=True
  )

  test_dataset = BERTDataset(tokenized_test, test_label)

  dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220407_inference/model'+str(i), config=config)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer = np.concatenate(output_pred).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label = pd.DataFrame(answer, columns=['index', 'label'])

  df_label.to_csv('/content/drive/MyDrive/220407_inference/pred_label'+str(i)+'.csv', index=False)

In [ ]:
print(my_variable)

In [ ]:
submission

In [ ]:
pred_label = pd.read_csv('/content/drive/MyDrive/220406_inference/pred_label.csv')

In [ ]:
pred_label

In [ ]:
def split_label(text:str):
  return text.split(' ')[0]

submission['digit_1'] = pred_label['label'].map(split_label)

def split_label(text:str):
  return text.split(' ')[1]

submission['digit_2'] = pred_label['label'].map(split_label)

def split_label(text:str):
  return text.split(' ')[2]

submission['digit_3'] = pred_label['label'].map(split_label)

In [ ]:
print(my_variable)

In [ ]:
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/220406_inference/submission.csv', index=False)